In [1]:
%cd ..
%pwd

/home/ssd5/zhanghui/DeepSpeech2.x


'/home/ssd5/zhanghui/DeepSpeech2.x'

In [2]:
import os
import time
import argparse
import functools
import paddle
import numpy as np

from deepspeech.utils.socket_server import warm_up_test
from deepspeech.utils.socket_server import AsrTCPServer
from deepspeech.utils.socket_server import AsrRequestHandler

from deepspeech.training.cli import default_argument_parser
from deepspeech.exps.deepspeech2.config import get_cfg_defaults

from deepspeech.frontend.utility import read_manifest
from deepspeech.utils.utility import add_arguments, print_arguments

from deepspeech.models.deepspeech2 import DeepSpeech2Model
from deepspeech.models.deepspeech2 import DeepSpeech2InferModel
from deepspeech.io.dataset import ManifestDataset


from paddle.inference import Config
from paddle.inference import create_predictor

from deepspeech.frontend.utility import read_manifest

2021-03-17 11:09:34,972 - WARNING - override cat of paddle.Tensor if exists or register, remove this when fixed!
2021-03-17 11:09:34,973 - WARNING - override size of paddle.Tensor if exists or register (`to_static` do not process `size` property, maybe some `paddle` api dependent on it), remove this when fixed!
2021-03-17 11:09:34,974 - WARNING - register user masked_fill to paddle.Tensor, remove this when fixed!
2021-03-17 11:09:34,975 - WARNING - register user masked_fill_ to paddle.Tensor, remove this when fixed!
2021-03-17 11:09:34,975 - WARNING - register user repeat to paddle.Tensor, remove this when fixed!
2021-03-17 11:09:34,976 - WARNING - register user glu to paddle.nn.functional, remove this when fixed!
2021-03-17 11:09:34,976 - WARNING - register user GLU to paddle.nn, remove this when fixed!
2021-03-17 11:09:34,977 - WARNING - register user ConstantPad2d to paddle.nn, remove this when fixed!
2021-03-17 11:09:34,977 - WARNING - override ctc_loss of paddle.nn.functional if e

In [3]:
print(paddle.__version__)
print(paddle.version)

0.0.0
<module 'paddle.version' from '/home/ssd5/zhanghui/DeepSpeech2.x/tools/venv-dev/lib/python3.7/site-packages/paddlepaddle_gpu-0.0.0-py3.7-linux-x86_64.egg/paddle/version.py'>


/home/ssd5/zhanghui/DeepSpeech2.x/tools/venv-dev/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
parser = default_argument_parser()
add_arg = functools.partial(add_arguments, argparser=parser)
add_arg('host_ip',          str,
        'localhost',
        "Server's IP address.")
add_arg('host_port',        int,    8086,    "Server's IP port.")
add_arg('speech_save_dir',  str,
        'demo_cache',
        "Directory to save demo audios.")
add_arg('warmup_manifest',  str, "examples/aishell/data/manifest.test", "Filepath of manifest to warm up.")
add_arg(
    "--model_file",
    type=str,
    default="examples/aishell/jit.model.pdmodel",
    help="Model filename, Specify this when your model is a combined model."
)
add_arg(
    "--params_file",
    type=str,
    default="examples/aishell/jit.model.pdiparams",
    help=
    "Parameter filename, Specify this when your model is a combined model."
)
add_arg(
    "--model_dir",
    type=str,
    default=None,
    help=
    "Model dir, If you load a non-combined model, specify the directory of the model."
)
add_arg("--use_gpu",type=bool,default=True, help="Whether use gpu.")
args = parser.parse_args("--checkpoint_path examples/aishell/ckpt/checkpoints/step-1876 --config examples/aishell/conf/deepspeech2.yaml --opts data.test_manifest examples/aishell/data/manifest.test data.mean_std_filepath examples/aishell/data/mean_std.npz  data.vocab_filepath examples/aishell/data/vocab.txt".split())


config = get_cfg_defaults()
if args.config:
    config.merge_from_file(args.config)
if args.opts:
    config.merge_from_list(args.opts)
config.freeze()
print(config)

args.warmup_manifest = config.data.test_manifest
print_arguments(args)

if args.dump_config:
    with open(args.dump_config, 'w') as f:
        print(config, file=f)

data:
  augmentation_config: conf/augmentation.config
  batch_size: 64
  dev_manifest: data/manifest.dev
  keep_transcription_text: False
  max_duration: 27.0
  max_freq: None
  mean_std_filepath: examples/aishell/data/mean_std.npz
  min_duration: 0.0
  n_fft: None
  num_workers: 0
  random_seed: 0
  shuffle_method: batch_shuffle
  sortagrad: True
  specgram_type: linear
  stride_ms: 10.0
  target_dB: -20
  target_sample_rate: 16000
  test_manifest: examples/aishell/data/manifest.test
  train_manifest: data/manifest.train
  use_dB_normalization: True
  vocab_filepath: examples/aishell/data/vocab.txt
  window_ms: 20.0
decoding:
  alpha: 2.6
  batch_size: 128
  beam_size: 300
  beta: 5.0
  cutoff_prob: 0.99
  cutoff_top_n: 40
  decoding_method: ctc_beam_search
  error_rate_type: cer
  lang_model_path: data/lm/zh_giga.no_cna_cmn.prune01244.klm
  num_proc_bsearch: 10
model:
  num_conv_layers: 2
  num_rnn_layers: 3
  rnn_layer_size: 1024
  share_rnn_weights: False
  use_gru: True
training:


In [5]:
dataset = ManifestDataset(
        config.data.test_manifest,
        config.data.vocab_filepath,
        config.data.mean_std_filepath,
        augmentation_config="{}",
        max_duration=config.data.max_duration,
        min_duration=config.data.min_duration,
        stride_ms=config.data.stride_ms,
        window_ms=config.data.window_ms,
        n_fft=config.data.n_fft,
        max_freq=config.data.max_freq,
        target_sample_rate=config.data.target_sample_rate,
        specgram_type=config.data.specgram_type,
        use_dB_normalization=config.data.use_dB_normalization,
        target_dB=config.data.target_dB,
        random_seed=config.data.random_seed,
        keep_transcription_text=True)

In [6]:
model = DeepSpeech2InferModel.from_pretrained(dataset, config,
                                             args.checkpoint_path)
model.eval()

/home/ssd5/zhanghui/DeepSpeech2.x/tools/venv-dev/lib/python3.7/site-packages/paddlepaddle_gpu-0.0.0-py3.7-linux-x86_64.egg/paddle/fluid/dygraph/layers.py:1303: UserWarning: Skip loading for decoder.ctc_lo.weight. decoder.ctc_lo.weight is not found in the provided dict.
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
/home/ssd5/zhanghui/DeepSpeech2.x/tools/venv-dev/lib/python3.7/site-packages/paddlepaddle_gpu-0.0.0-py3.7-linux-x86_64.egg/paddle/fluid/dygraph/layers.py:1303: UserWarning: Skip loading for decoder.ctc_lo.bias. decoder.ctc_lo.bias is not found in the provided dict.
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
2021-03-17 11:10:00,017 - INFO - [checkpoint] Rank 0: loaded model from examples/aishell/ckpt/checkpoints/step-1876.pdparams


layer summary:
encoder.conv.conv_in.conv.weight|[32, 1, 41, 11]|14432
encoder.conv.conv_in.bn.weight|[32]|32
encoder.conv.conv_in.bn.bias|[32]|32
encoder.conv.conv_in.bn._mean|[32]|32
encoder.conv.conv_in.bn._variance|[32]|32
encoder.conv.conv_stack.0.conv.weight|[32, 32, 21, 11]|236544
encoder.conv.conv_stack.0.bn.weight|[32]|32
encoder.conv.conv_stack.0.bn.bias|[32]|32
encoder.conv.conv_stack.0.bn._mean|[32]|32
encoder.conv.conv_stack.0.bn._variance|[32]|32
encoder.rnn.rnn_stacks.0.fw_fc.weight|[1312, 3072]|4030464
encoder.rnn.rnn_stacks.0.fw_bn.weight|[3072]|3072
encoder.rnn.rnn_stacks.0.fw_bn.bias|[3072]|3072
encoder.rnn.rnn_stacks.0.fw_bn._mean|[3072]|3072
encoder.rnn.rnn_stacks.0.fw_bn._variance|[3072]|3072
encoder.rnn.rnn_stacks.0.bw_fc.weight|[1312, 3072]|4030464
encoder.rnn.rnn_stacks.0.bw_bn.weight|[3072]|3072
encoder.rnn.rnn_stacks.0.bw_bn.bias|[3072]|3072
encoder.rnn.rnn_stacks.0.bw_bn._mean|[3072]|3072
encoder.rnn.rnn_stacks.0.bw_bn._variance|[3072]|3072
encoder.rnn.rnn_st

In [11]:
def init_predictor(args):
    if args.model_dir is not None:
        config = Config(args.model_dir)
    else:
        config = Config(args.model_file, args.params_file)

    config.enable_memory_optim()
    if args.use_gpu:
        config.enable_use_gpu(memory_pool_init_size_mb=1000, device_id=0)
    else:
        # If not specific mkldnn, you can set the blas thread.
        # The thread num should not be greater than the number of cores in the CPU.
        config.set_cpu_math_library_num_threads(4)
        #config.enable_mkldnn()
        
    config.switch_ir_optim(False)

    predictor = create_predictor(config)
    return predictor

def run(predictor, audio, audio_len):
    # copy img data to input tensor
    input_names = predictor.get_input_names()
    for i, name in enumerate(input_names):
        print("input:", i, name)
        
    audio_tensor = predictor.get_input_handle('audio')
    audio_tensor.reshape(audio.shape)
    audio_tensor.copy_from_cpu(audio.copy())
    
    audiolen_tensor = predictor.get_input_handle('audio_len')
    audiolen_tensor.reshape(audio_len.shape)
    audiolen_tensor.copy_from_cpu(audio_len.copy())

    output_names = predictor.get_output_names()
    for i, name in enumerate(output_names):
        print("output:", i, name)

    # do the inference
    predictor.run()

    results = []
    # get out data from output tensor
    output_names = predictor.get_output_names()
    for i, name in enumerate(output_names):
        output_tensor = predictor.get_output_handle(name)
        output_data = output_tensor.copy_to_cpu()
        results.append(output_data)

    return results


predictor = init_predictor(args)

def file_to_transcript(filename):
        print(filename)
        feature = dataset.process_utterance(filename, "")
        audio = np.array([feature[0]]).astype('float32')  #[1, D, T]
        audio_len = feature[0].shape[1]
        audio_len = np.array([audio_len]).astype('int64')  # [1]
        
        
        i_probs = run(predictor, audio, audio_len)
        print('jit:', i_probs[0], type(i_probs[0]))
        
        audio = paddle.to_tensor(audio)
        audio_len = paddle.to_tensor(audio_len)
        print(audio.shape)
        print(audio_len.shape)
        
        #eouts, eouts_len = model.encoder(audio, audio_len)
        #probs = model.decoder.probs(eouts)
        probs = model.forward(audio, audio_len)
        print('paddle:', probs.numpy())
        
        flag = np.allclose(i_probs[0], probs.numpy())
        print(flag)
        
        return probs

#         result_transcript = model.decode(
#             audio,
#             audio_len,
#             vocab_list=dataset.vocab_list,
#             decoding_method=config.decoding.decoding_method,
#             lang_model_path=config.decoding.lang_model_path,
#             beam_alpha=config.decoding.alpha,
#             beam_beta=config.decoding.beta,
#             beam_size=config.decoding.beam_size,
#             cutoff_prob=config.decoding.cutoff_prob,
#             cutoff_top_n=config.decoding.cutoff_top_n,
#             num_processes=config.decoding.num_proc_bsearch)
#         return result_transcript[0]

In [12]:
manifest = read_manifest(args.warmup_manifest)

for idx, sample in enumerate(manifest[:1]):
    print("Warm-up Test Case %d: %s", idx, sample['audio_filepath'])
    start_time = time.time()
    transcript = file_to_transcript(sample['audio_filepath'])
    finish_time = time.time()
#     print("Response Time: %f, Transcript: %s" %
#           (finish_time - start_time, transcript))
    break

Warm-up Test Case %d: %s 0 /home/ssd5/zhanghui/DeepSpeech2.x/examples/aishell/../..//examples/dataset/aishell/data_aishell/wav/test/S0764/BAC009S0764W0121.wav
/home/ssd5/zhanghui/DeepSpeech2.x/examples/aishell/../..//examples/dataset/aishell/data_aishell/wav/test/S0764/BAC009S0764W0121.wav
input: 0 audio
input: 1 audio_len
output: 0 tmp_75
jit: [[[1.40282078e-04 3.31296207e-04 5.57157793e-04 ... 1.07916087e-04
   8.73636964e-05 1.96113906e-04]
  [1.38032061e-04 2.70526099e-04 4.53807996e-04 ... 1.02293277e-04
   8.40202629e-05 1.90612729e-04]
  [1.38912103e-04 2.45687814e-04 3.99624696e-04 ... 9.70420660e-05
   7.88255784e-05 1.80084753e-04]
  ...
  [3.28999187e-04 2.59723864e-04 3.03535169e-04 ... 2.82066030e-04
   1.11002744e-04 1.27009131e-04]
  [2.91427423e-04 2.20203598e-04 2.85082555e-04 ... 3.27318383e-04
   1.09202861e-04 1.17112293e-04]
  [3.63971514e-04 1.47859042e-04 2.24457763e-04 ... 3.63016297e-04
   1.34765272e-04 1.61947115e-04]]] <class 'numpy.ndarray'>
[1, 161, 419]
[

In [13]:
def test(filename):
    feature = dataset.process_utterance(filename, "")
    audio = np.array([feature[0]]).astype('float32')  #[1, D, T]
    audio_len = feature[0].shape[1]
    audio_len = np.array([audio_len]).astype('int64')  # [1]
    
    print(audio.shape, audio_len.shape)

    i_probs = run(predictor, audio, audio_len)
    print('jit:', i_probs[0])
    return i_probs
    
probs = test(sample['audio_filepath'])

(1, 161, 419) (1,)
input: 0 audio
input: 1 audio_len
output: 0 tmp_75
jit: [[[1.40282078e-04 3.31296207e-04 5.57157793e-04 ... 1.07916087e-04
   8.73636964e-05 1.96113906e-04]
  [1.38032061e-04 2.70526099e-04 4.53807996e-04 ... 1.02293277e-04
   8.40202629e-05 1.90612729e-04]
  [1.38912103e-04 2.45687814e-04 3.99624696e-04 ... 9.70420660e-05
   7.88255784e-05 1.80084753e-04]
  ...
  [3.28999187e-04 2.59723864e-04 3.03535169e-04 ... 2.82066030e-04
   1.11002744e-04 1.27009131e-04]
  [2.91427423e-04 2.20203598e-04 2.85082555e-04 ... 3.27318383e-04
   1.09202861e-04 1.17112293e-04]
  [3.63971514e-04 1.47859042e-04 2.24457763e-04 ... 3.63016297e-04
   1.34765272e-04 1.61947115e-04]]]


In [10]:
a = paddle.to_tensor([1,3,4])
a.numpy?

/home/ssd5/zhanghui/DeepSpeech2.x/tools/venv-dev/lib/python3.7/site-packages/paddlepaddle_gpu-0.0.0-py3.7-linux-x86_64.egg/paddle/tensor/creation.py:143: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
